<span type="title">策略模式、观察者模式和装饰器模式</span> | <span type="update">2018-08-31</span> - Version <span type="version">1.1</span>
    
    
<span type="intro"><p class="card-text">本章首先介绍了面向对象的基础：抽象、封装、多态和继承，之后提到了四大设计原则：区分变化和不变的部分，针对接口而不是实现变成，多用组合而不是继承，类和类之间尽可能解耦。好的设计应该满足复用、可扩展、可维护三大特征。</p><p class="card-text">在第二部分讲解了策略模式：策略模式指的是定义算法族，封装它，然后使其可互相替代，之后将可变的算法和算法使用者区别开。这种设计能够满足我们希望将代码从方法中分离出去到状态，然后区分可变和不可变的这一目的。</p><p class="card-text">第三部分主要讲解了观察者模式，观察者模式定义了对象的一对多关系，当一个对象的状态改变，会自动通知并且更新其余对象的状态（通过接口调用更新方法），这样实现了轻度的耦合，同时满足了我们希望将可变的状态跨类别集中管理的目的。</p><p class="card-text">在第四部分主要讲解了装饰器模式，这种模式指的是动态的将责任添加到对象上。在实现层面，其通过内部实例在构造时传入和保存对象，然后通过继承以达成和这个对象共享同一套API和类名称的目的，之后操纵此对象完成对API的服务，并且在此过程中添加装饰。装饰器模式主要适用于不定个变化需要动态添加的情况，这点和策略模式很像（策略模式强调相互替代的变化动态添加）</p></span>

# 面向对象导论

## 面向对象基础

抽象、封装、多态、继承

其中封装是对于实例变量而言的，其保存了对象的状态。多态是由于继承复用代码带来的一种调用超类的接口的契约保证。

## 面向对象设计原则

设计模式的任务是设计出弹性的，可以维护的，能够应对变化的代码，这仅靠抽象，继承，多态这些OOP观念是不足以做到的。

所有的设计模式都遵循以下的设计原则：

- 找出应用中可能需要变化之处，把它们独立出来，不要和那些不需要变化的代码混在一起。换句话说，封装变化。
- 针对接口（超类型）编程，而不是针对实现编程
- 多用组合，少用继承（组合可以将代码被封装成类，可以在运行时动态改变行为）
- 类和类之间尽可能的解耦，使用尽可能少的方法传递信息
- 开闭原则：面对修改关闭，面对扩展开放

良好的面向对象设计应该满足 复用（使用组合或者继承）、可扩充（扩充接口）、可维护（多态，针对接口编程）三大特征。

# 策略模式

**策略模式定义了算法族，分别封装起来，让它们之间可以互相替换，此模式让算法的变化独立于使用算法的客户。**

策略模式本身就是组合，其对应has-a关系，使用实例变量来保存状态，针对接口编程来调用和更改这个变量以改变状态。JDBC就是典型的策略模式。

# 观察者模式

**观察者模式定义了对象之间的一对多依赖，当一个对象改变状态时，它的所有依赖者都会收到通知并且自动更新。**主题->观察者 = 发布者->订阅者模式，一个发布者，多个订阅者，订阅者可以选择进入或者退出发布者名单，这样就不会收到更新。

常见的观察者模式是GUI，JavaBeans，RMI

## 自己实现观察者模式

常用Subject和Observer表示类名。

其中Subject应该保存一个注册者列表，提供添加和删除注册者的方法，提供更新注册者数据的方法，并且将自己的状态通过注册者的接口传递给注册者。

对于注册者而言，应该提供一个接受发布者状态的接口，应该有保存这些状态的实例，同时最好有发布者的一个实例，方便和发布者绑定或者取消订阅。

**几乎完全解耦 **观察者模式类似于集体主义，一个对象保留并且获取状态，而其余对象则建立和这个对象的联系，并且从其中获取状态。由于两者间传递信息仅使用了注册者update接口，除此之外，发布者保存了含有注册者引用的对象，而这个对象的保存是依靠注册者传入发布者的一个实例，调用add添加的，因此除了注册者update，发布者的add，remove之外，完全做到了解耦。发布者自动进入注册者内存并且更新其状态，这个过程不需要注册者任何操作。

## 使用util实现观察者模式

```java
import java.util.Observable;
import java.util.Observer;

public class Main {
    public static void main(String[] args) {
        Server server = new Server();
        ClientA clientA = new ClientA(server);
        server.setStatic(10.0,23.0);
        server.notice();
    }
}
class Server extends Observable {
    private double temp;
    private double pressure;
    public double getTemp() {
        return temp;
    }
    public double getPressure() {
        return pressure;
    }
    public void setStatic(double temp, double pressure) {
        this.temp = temp;
        this.pressure = pressure;
    }
    public void notice() {
        this.setChanged();
        this.notifyObservers();
    }
}
class ClientA implements Observer {
    private double temp;
    private double pressure;
    private Observable server;
    ClientA(Observable server) {
        this.server = server;
        this.server.addObserver(this);
    }
    @Override
    public void update(Observable o, Object arg) {
        temp = ((Server)o).getTemp();
        pressure = ((Server)o).getPressure();
        display();
    }
    public void display() {
        System.out.println("Current temp is " + temp + ". Current Pressure is " + pressure);
    }
}
```

上述代码是使用观察者的一个例子，类似的，在Swing以及JavaFx中，存在大量的这种观察者设计模式：listener。

注意上述的惯用方法，对于观察对象而言，其主做的就是从自身保有的列表中获取观察者，然后调用Observaer的update方法，将自己传递过去。我们主要的任务就是实现一些需要在传送之后用于获取Observable状态的方法，**getXXX()**，然后**setChanged并且notifyObservers**即可。

对于观察者而言，我们需要选择一个观察对象，**注册它**或者从它的名单中删除。此外，在**update方法**中决定当收到观察对象更新时的自己的操作。除此之外，不需要做任何事情，当观察对象有更新，其会自动调用update改变观察者的状态。

## 观察者模式的设计原则

- 封装变化和不变的部分：观察者模式中，变化的部分是Observable对象，因此将其单独封装为一个类，然后将所有Observer的这些状态抽取出来，接受更新
- 针对接口而不是实现编程：我们使用了update方法来更新观察者状态，使用addObserver来进行观察对象的注册。需要注意，可以寻求对于getXXXX这种需要在观察者update方法进行实现的接口进行标准化，使用interface达成契约。这样的话，Observer只和Observable以及这个获取具体状态的契约接口耦合。Observable只和这个Observer的update方法耦合。
- 多用组合，少用继承：没错，我们在Observer中保存了Observable的实例，用于注册和解除注册。彼此状态的传递使用的也是实例方法，而不是继承。
- 类和类尽量解耦：同2.

# 装饰器模式

装饰器模式和策略模式很像，其都是解决多个互相替代的类的可变和不可变区分的问题。对于策略模式而言，使用实例对象来动态保存和修改主要类的算法，这些算法是互相替代的，使用一个共有接口。而装饰器模式面临的情况更加复杂，装饰器模式需要解决不定个数的类的组合，这显然和“互相替代的算法”是冲突的。

装饰器模式指的是：**动态的将责任附加到对象上**。想要扩展功能，装饰器模式提供了区别于继承的另外一种选择。这种选择很好的满足了**面对扩展开放，面对修改关闭的设计原则**。在实现层面，装饰器模式通过保存一个内部实例，然后通过同一套API来操纵这套实例的方法来完成伪装和扩展。

装饰器模式实现的方法是：不改变被装饰对象的代码，首先找到被装饰对象被调用的父类（提供了调用被装饰对象的API方法），然后使用一个抽象装饰器类去继承这个被装饰对象的父类以获得相同的类型信息和API接口（而不是为了复用代码）。之后，对于这个装饰器抽象类编写不同的装饰器，这些装饰器内部保存被装饰对象的实例，一般通过构造器传入和设置。之后，因为装饰器间接实现了被装饰器的API和类型，因此，可以直接像使用被装饰对象一样使用装饰器。程序员需要做的是，对于装饰器的这些方法进行重载，一般而言，调用其内部保存的实例（被装饰对象）的相同API，然后添加一点别的东西进去。这样的话，我们就可以像使用被装饰对象一样使用装饰器，因为它们共享同样的API和类型信息。

举例如下：

## 装饰器模式用于装饰咖啡配料

```java
package com.mazhangjing.study;
import java.io.*;

abstract class Coffee {
    protected String description = "No description";
    public String getDescription() {
        return description;
    }
    abstract public double cost();
}
class NullCoffee extends Coffee {
    @Override
    public String getDescription() {
        return "Null Coffee";
    }
    @Override
    public double cost() {
        return 0;
    }
}
class HouseBlend extends Coffee {
    HouseBlend() { this.description = "HouseBlend"; }
    @Override
    public double cost() {
        return 10.0;
    }
}
class DarkRoast extends Coffee {
    DarkRoast() { this.description = "DarkRoast"; }
    @Override
    public double cost() {
        return 16.0;
    }
}
class Espress extends Coffee {
    Espress() { this.description = "Espress"; }
    @Override
    public double cost() {
        return 14.0;
    }
}
class Decaf extends Coffee {
    Decaf() { this.description = "Decaf"; }
    @Override
    public double cost() {
        return 8.0;
    }
}
abstract class CondimentDecorator extends Coffee {
    protected Coffee coffee = new NullCoffee();
    //这里放置coffee其实是可有可无的，如果没有，那么就要在装饰器子类自己保存实例对象，繁琐但自由度更高。这里使用空对象
    //也是可有可无的，因为我们使用装饰器一般在构造器中创建实例，因此可以避免空对象的问题
    //装饰对象必须装饰一个元素，装饰对象和这个元素同族，但仅仅是为了类型兼容，没有重用任何方法

    //这里abstract描述的原因是，因为我们想要添加过配件的咖啡完整描述自己
    public abstract String getDescription();
}
class Mocha extends CondimentDecorator {
    Mocha(Coffee coffee) { this.coffee = coffee; }
    @Override
    public double cost() {
        return 2.0 + coffee.cost();
    }
    @Override
    public String getDescription() {
        return coffee.getDescription() + ", Mocha";
    }
}
class Soy extends CondimentDecorator {
    Soy(Coffee coffee) { this.coffee = coffee; }
    @Override
    public double cost() {
        return coffee.cost() + 1.0;
    }
    @Override
    public String getDescription() {
        return coffee.getDescription() + ", Soy";
    }
}
class Whip extends CondimentDecorator {
    Whip(Coffee coffee) { this.coffee = coffee; }
    @Override
    public double cost() {
        return coffee.cost() + 1.0;
    }
    @Override
    public String getDescription() {
        return coffee.getDescription() + ", Whip";
    }
}
public class DecoratorDemo {
    static void getOrder(Coffee...coffees) {
        for (Coffee coffee : coffees) {
            System.out.println("Order:\n" + coffee.getDescription() + ", Cost RMB" + coffee.cost());
        }
    }
    public static void main(String[] args) {
        Coffee order1 = new Whip(
                        new Soy(new Soy(
                        new Mocha(new HouseBlend()))));
        Coffee order2 = new Soy(new Whip(new DarkRoast()));
        Coffee order3 = new Mocha(new Decaf());
        Coffee order4 = new Espress();
        Coffee order5 = new Soy(new NullCoffee());
        getOrder(order1,order2,order3,order4,order5);
    }
}
```
## 装饰器模式用于过滤InputStream

一个实际的例子，是Java的IO对象，这里使用了过滤器，过滤掉大写B字母：

```java
class FuckBFilter extends FilterInputStream {
    FuckBFilter(InputStream in) {
        super(in);
    }
    @Override
    public int read() throws IOException {
        int c = super.read();
        if ((char)c == 'B') { c = (int)'b'; }
        return c;
    }
    public static void main(String[] args) throws Exception{
        FilterInputStream inputStream =
                new FuckBFilter(new FileInputStream("hello.txt"));
        int temp;
        while ((temp = inputStream.read()) != -1) {
            System.out.print((char)temp);
        }
    }
}
```

## 面向修改关闭，面向扩展开放

装饰器模式很好的说明了开闭原则，面对修改关闭，面对扩展开放，其通过保存一个内部实例，然后通过同一套API来操纵这套实例的方法来完成伪装和扩展的。但是需要注意：对于极端变化的情况使用开闭原则，而不要对任意代码使用开闭原则，否则会引入过多抽象层次，难以维护代码（双刃剑）。

观察者模式也提供了这样一种可以动态改变状态而不需要手动操作的能力（通过观察者更新的接口改变其状态）。此外，观察者模式还提供了动态改变观察者列表，提供扩展的能力，而完全无需修改观察对象本身的代码（通过在观察者构造时调用观察对象添加的接口）

开闭原则是面向对象设计原则的第五条：变化，接口，组合，解耦，开闭。

## 装饰器的适用和缺陷

适用：不定个变化需要运行时添加，区别于策略模式的算法互相替代性。在这种情况下，我们通过一连串的嵌套构造来组合变化，由于其共享一套API，因此不论何种变化，接口都是不变的。

缺陷：装饰器种类繁多，此外，如果我们找不到一个接口来让装饰器共享，或者错误的将装饰器构造成其本身类型（忘记转型和多态的情况下），会造成地狱级别的代码。尤其是这种类型被应用在方法参数或者返回值中。

——————————————————————-

最后修改：

2018年8月31日 添加策咯模式，观察者模式

2018年9月1日 添加装饰器模式